# Spectra Reparametrization

We calculate and visualize the reconstruction of stellar spectra with and without recasting to same physical parameters.

In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from scipy import spatial
import sys
import pickle

sys.path.insert(0,'/share/splinter/ddm/taggingClean/')

from tagging.src.datasets import ApogeeDataset
from tagging.src.networks import ConditioningAutoencoder,Embedding_Decoder,Feedforward


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## We define some hyperparameters

In [ ]:
n_batch = 9 
n_bins = 7751 
n_conditioned = 2

The next line is only used for getting the wavelength axis correct

In [ ]:
nlam = 8575 
start_wl = 4.179 
diff_wl = 6e-06 
val = diff_wl*(nlam) + start_wl  
wl_full_log = np.arange(start_wl,val, diff_wl) 
wl_full = [10**aval for aval in wl_full_log] 
xdata = np.array(wl_full)  
with open("../data/s_train_merged","rb") as f: 
    data = pickle.load(f)  
xdata = xdata[np.nonzero(np.nan_to_num(data[0]["spectra"]))[0]] 


We next load up the data. ```data_plotting``` contains the spectra we want to visualize but because w use data-driven normalization to not mess up normalization we append it to the original data```data``` 

In [ ]:
data = pd.read_pickle("/share/splinter/ddm/taggingClean/data/final/train/spectra_noiseless.pd")
data_plotting = pd.read_pickle("../data/plotting_merged.pd")

overall_data = data_plotting.append(data,ignore_index=True)

In [ ]:
dataset = ApogeeDataset(overall_data,n_bins)
evaluation_loader = torch.utils.data.DataLoader(dataset = dataset[0:18],
                                     batch_size = n_batch,
                                     shuffle = False,
                                     drop_last=True)